# Model Save & Restore
- 어플리케이션을 개발하는 것과 마찬가지로 텐서플로우로 학습 모델을 만들때에도 운영 환경을 고려하게 된다.
- 즉, 실제 데이터를 가지고 예측 모델을 만드는 과정을 거쳐서 나온 예측 함수를 매번 새로 학습시키는 것이 아니라 만들어진 모델에 맞는 데이터를 저장해서 그냥 쓰게 만드는 것이다.
- 이 예제는 기존에 학습하고 저장한 모델 정보를 불러와서 운영 환경에서 사용할 수 있게 모델을 복원하는 역할을 한다.

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
input_data = [[1,5,3,7,8,10,12],
              [5,8,10,3,9,7,1]]
label_data = [[0,0,0,1,0],
              [1,0,0,0,0]]

In [4]:
INPUT_SIZE = 7
HIDDEN1_LAYER_SIZE = 10
HIDDEN2_LAYER_SIZE = 8
CLASSES = 5
LEARNING_RATE = 0.5

- 데이터를 텐서로 변환하는 작업을 진행한다.
- 텐서에서 shape 의 첫번째는 배치사이즈로 보통 정의한다.
- 보통 배치 사이즈를 사전에 알기는 어렵기 때문에 None 으로 정의하기도 한다.

In [5]:
x = tf.placeholder( tf.float32, shape=[None, INPUT_SIZE])
y_ = tf.placeholder( tf.float32, shape=[None, CLASSES])

In [6]:
tensor_map  = {x: input_data, y_:label_data}

https://www.tensorflow.org/api_docs/python/tf/truncated_normal

The generated values follow a normal distribution with specified mean
and standard deviation, except that values whose magnitude is more
than 2 standard deviations from the mean are dropped and re-picked.

https://www.tensorflow.org/api_docs/python/tf/zeros

In [9]:
weight_hidden1 = tf.Variable( tf.truncated_normal( shape=[INPUT_SIZE, HIDDEN1_LAYER_SIZE]
, dtype=tf.float32) ) #
bias_hidden1 = tf.Variable( tf.zeros([HIDDEN1_LAYER_SIZE]), dtype=tf.float32)

weight_hidden2 = tf.Variable( tf.truncated_normal( shape=[HIDDEN1_LAYER_SIZE, HIDDEN2_LAYER_SIZE], dtype=tf.float32) )
bias_hidden2 = tf.Variable( tf.zeros([HIDDEN2_LAYER_SIZE]), dtype=tf.float32)

weight_output = tf.Variable( tf.truncated_normal( shape=[HIDDEN2_LAYER_SIZE, CLASSES], dtype=tf.float32) ) #
bias_output = tf.Variable( tf.zeros([CLASSES]), dtype=tf.float32)

In [10]:
parameter_list = [weight_hidden1, bias_hidden1, weight_hidden2, bias_hidden2, weight_output, bias_output]
saver = tf.train.Saver(parameter_list)

In [13]:
hidden1 = tf.sigmoid(tf.matmul(x, weight_hidden1) + bias_hidden1)
hidden2 = tf.sigmoid(tf.matmul(hidden1, weight_hidden2) + bias_hidden2)
y = tf.sigmoid(tf.matmul(hidden2, weight_output) + bias_output)

In [19]:
sess = tf.Session()

# 기존의 방식대로 초기값 상태로 예측을 진행하면 정상적인 결과를 얻어오지 못한다.

In [26]:
sess.run( tf.global_variables_initializer() )
result = sess.run( y, tensor_map )
print result


[[ 0.55549502  0.61399895  0.39525995  0.58042097  0.77931082]
 [ 0.54080635  0.59290749  0.45076743  0.53675801  0.76859021]]


- https://www.tensorflow.org/api_docs/python/tf/train/Saver

In [27]:
# sess.run( tf.global_variables_initializer() )
saver.restore(sess, './tensorflow.ckpt')
result = sess.run( y, tensor_map )
print result


[[  3.03515606e-03   6.88230793e-04   5.21090406e-04   9.96888816e-01
    3.42063780e-04]
 [  9.97143924e-01   4.23800724e-04   6.04767876e-04   2.94184452e-03
    7.67644146e-04]]


- 체크포인트 값을 로드하는 과정중에 에러가 발생한다면 대부분의 경우는 텐서의 이름이 저장할때와 다르기 때문에 발생하는 경우가 많다.